## Mục đích mà bài toán hướng tới và phương hướng giải quyết
---
- Bài toán: Dự đoán số điểm đánh giá một bộ phim trên thang điểm 0 tới 10 dựa trên IMDB_Rating.
- Mục tiêu:
    - Bài toán chung đặt ra bao gồm:
        - Crawl dữ liệu các phim và trích xuất các dữ liệu cần thiết để có thể dự đoán bộ phim có điểm số bao nhiêu.
        - Minh họa dữ liệu để xem các đặt trưng của dữ liệu đó thế nào để phục vụ cho công việc tiếp theo.
        - Phân tích các đặt trưng dựa EDA và thực hiện các Feature Engineering với các hành động như bổ sung đặc trưng bị trống, biến đổi (Label hóa) dữ liệu về dạng số để thực hiện bài toán dự đoán.
        - Dự đoán điểm bộ phim đó như thế nào dựa trên các đặc trưng mà bộ phim có.

<h1>1. Crawl data</h1>

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import numpy as np
import copy
import pandas as pd

Trang web mà chúng ta sẽ sử dụng để crawl dữ liệu: https://reelgood.com/movies <br /> <br />
<img style="display: block; width: 80%; object-fit: cover; margin: auto" src="https://res.cloudinary.com/toanil315/image/upload/v1656231528/Screenshot_2712_pbnhjv.png" />

Đầu tiên chúng ta sẽ tiến hành lấy link của các bộ phim. Chúng ta sẽ tiến hành lấy dữ liệu phim ở trang detail của chúng để đảm bảo có đủ thông tin nhất có thể. Chúng ta vẫn có thể lấy dữ liệu ở trang list nhưng sẽ thiếu các thông tin (description, director, genre...) <br /> <br />
<img style="display: block; width: 80%; object-fit: cover; margin: auto" src="https://res.cloudinary.com/toanil315/image/upload/v1656231615/Screenshot_2713_t0khcw.png" />

In [ ]:
#link of films in reelgood.com
for i in range(0, 5500, 50):
    URL= "https://reelgood.com/movies?offset=" + str(i)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    f = open("movies.txt", "a")
    for link in soup.select('[itemprop=itemListElement] [itemprop=url]'):
        data = link.get('content')
        f.write(data)
        f.write("\n")

Tiến hành tạo ra một object mẫu của film

In [ ]:
filmObj = {
      "Title": None,
      "IMDB_Rating": None,
      "Reelgood_Rating": None,
      "Genres": [],
      "Year": None,
      "Tags": [],
      "Country": None,
      "Source": None,
      "Rated": None,
      "Duration": None,
      "Description": None,
      "Director": None,
      "Employees": [],
}

Tiến hành lấy dữ liệu bằng cách request tới link của các bộ phim trong tập chúng ta vừa lấy được lúc nãy

In [ ]:
with open('movies.txt') as f:
    links = [line.rstrip() for line in f]

In [ ]:
arr_film = []
for i in range(0, len(links)):
  link = links[i]
  film = copy.deepcopy(filmObj)

  try:
    with requests.get(link) as page:
      soup = BeautifulSoup(page.content, 'html.parser')
      film['Title'] = soup.find(class_="css-of585f e14injhv7")
      if film['Title'] != None:
        film['Title'] = film['Title'].get_text()

      div_info = soup.find(class_="css-1su90nd ey4ir3j0")

      # GET SCORE
      film['IMDB_Rating'] = div_info.find(class_="css-xmin1q ey4ir3j3").get_text()
      film['Reelgood_Rating'] = div_info.find(class_="css-xmin1q ey4ir3j8")
      if film['Reelgood_Rating'] != None:
        film['Reelgood_Rating'] = film['Reelgood_Rating'].get_text()

      # GET INFO BY TAG A
      list_info = div_info.find_all(class_="css-10wrqt0")
      for info in list_info:
        type_info = info.get("href")
        if('/genre' in type_info):
          film['Genres'].append(info.get_text())
        elif '/year' in type_info:
          film['Year'] = info.get_text()
        elif '/list' in type_info: 
          film['Tags'].append(info.get_text())
        elif '/origin' in type_info:
          film['Country'] = info.get_text()
        elif '/source' in type_info:
          film['Source'] = info.get_text()

      # GET INFO BY TAG META
      film['Rated'] = div_info.find('meta', itemprop='contentRating')
      if film['Rated'] != None:
        film['Rated'] = film['Rated'].get("content", None)
      film['Duration'] = div_info.find('meta', itemprop='duration')
      if film['Duration'] != None: 
        film['Duration'] = film['Duration'].get("content", None)

      # GET DESCRIPTION
      film['Description'] = soup.find('p', itemprop="description").get_text()

      # GET CAST & CREW
      list_employee = soup.find_all('h3', itemprop="name", class_="css-11yr18h egg5eqo2")
      # GET DIRECTOR
      if len(list_employee) > 0:
        film['Director'] = list_employee[0].get_text()
        #GET EMPLOYEE
        for i in range(1, len(list_employee)):
          employee = list_employee[i]
          film['Employees'].append(employee.get_text())

      print(link, " done")
      arr_film.append(film)

  except (NameError, AttributeError) as e:
    print(e)
    continue
f.close()

tiến hành convert thành dataframe và lưu dưới dạng file csv
<p >
    <span style="text-decoration: underline">Lưu ý</span>: <br />
    Khi request thì có một số link bị hỏng hoặc tiêu đề của film được đánh class sai, do đó sẽ xuất hiện tình trạng không thể lấy dữ liệu và request timeout gây ra vấn đề bị ngắt khi đang lấy. Chúng ta có thể xử lý bằng cách crawl thành từng file nhỏ và ghép chúng lại thành 1 dataframe tổng sau khi đã chạy hết links đã lấy
</p>


In [ ]:
df = pd.DataFrame(arr_film)
df.to_csv('film_reelgood.csv',index=False, encoding="utf-8")

Chúng ta có thể để ý rằng trong đặc trưng Description, thường ở phần cuối nó sẽ có số luợng người đánh giá bộ phim, có thể đặc trưng này sẽ quan trọng nên chúng ta cần tiến hành trích xuất nó ra

In [ ]:
df_film = pd.read_csv("film_reelgood.csv", encoding="utf-8")
df_film.loc[0]['Description']

In [ ]:
arr_votes = []

# extract user votes
for i in df_film['Description']:
    sub_str = i[-50:]
    startIndex = sub_str.find("(")
    endIndex = sub_str.find(")")
    if('votes' in sub_str[startIndex: endIndex]):
        arr_votes.append(sub_str[startIndex + 1: endIndex])
    else: arr_votes.append(np.NaN)

# concat into df and save as file csv
arr_votes = np.array(arr_votes)
df_votes = pd.DataFrame(arr_votes, columns = ['User_Votes'])
result = pd.concat([df_film, df_votes], axis=1)
result.to_csv("raw_data.csv", encoding="utf-8")

Kết quả cuối cùng:

In [ ]:
df_film = pd.read_csv("raw_data.csv", encoding="utf-8")
df_film.info()
df_film.head(2)